note: 这个代码是为了测试sentence embedding model的效果

## 1 训练sentence embedding model，将元组转换为向量

### 1.1 加载数据，取出无关元组

In [2]:
import pandas as pd
import numpy as np
import string
import math
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")  #忽略告警
import os

import torch
from sentence_transformers import util
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [3]:
# 加载数据
file_path = 'data/Dirty/DBLP-Scholar/'
table_a_df = pd.read_csv(file_path + 'tableA.csv')
table_b_df = pd.read_csv(file_path + 'tableB.csv')

train_df = pd.read_csv(file_path + 'train.csv')
valid_df = pd.read_csv(file_path + 'valid.csv')
test_df = pd.read_csv(file_path + 'test.csv')

In [4]:
table_a_df.head()

,id,title,authors,venue,year
0,0,towards a cooperative transaction model - the ...,NaN,vldb,NaN
1,1,sql/xml is making good progress sigmod record ...,"a eisenberg , j melton",NaN,NaN
2,2,using formal methods to reason about semantics...,"p ammann , s jajodia , i ray",NaN,1995.0
3,3,editor 's notes l liu 2002,NaN,sigmod record,NaN
4,4,report on the acm fourth international worksho...,NaN,NaN,NaN


In [5]:
train_df.head()

,ltable_id,rtable_id,label
0,2563,6425,0
1,1861,64163,1
2,2187,64008,1
3,611,15632,0
4,631,27684,0


In [6]:
#提取字符串当中的英文字母、浮点数、整数，将其他字符替换为空格
def DataPeocess(attr_value):
    '''
    对输入的文本，提取当中的英文字母、浮点数、整数，将其他字符替换为空格
    '''
    s_list = list(str(attr_value))
    max_len = len(s_list)
    for i in range(len(s_list)):
        if s_list[i] == '.':
            if i < max_len-1 and i > 0 and s_list[i+1].isdigit() and s_list[i-1].isdigit():
                None
            else :
                s_list[i] = ' '
        elif not s_list[i].isalnum():
            s_list[i] = ' '
        else:
            s_list[i] == ' '

    return ''.join(s_list)

def ExtractData(df):
    '''
    对输入的dataFrame的所有属性值，去除无关字符，然后全部转换为小写
    '''
    attr_list = df.columns.values.tolist()
    attr_list.remove('id')

    #首先，先对每个属性的文本去除无关符号，将单词用空格分开
    for attr in attr_list:
        temp_w_num = 0

        for i in range(df.shape[0]):
            temp_value = DataPeocess(df.loc[i,attr])
            # 将所有英文字符转换为小写的
            df.loc[i,attr] = temp_value.lower()

    return df

# 去除原始数据当中的无关符号，将其替换为空格
table_a_df = ExtractData(table_a_df)
table_b_df = ExtractData(table_b_df)

In [7]:
#计算全局数据中，每个属性对应的最大单词数量
attr_maxnum_d = dict()

attr_list = table_a_df.columns.values.tolist()
attr_list.remove('id')

for attr in attr_list:
    temp_num = 0

    for i in range(table_a_df.shape[0]):
        temp_value = table_a_df.loc[i, attr]
        temp_num = max(temp_num, len(temp_value.split()))

    for i in range(table_b_df.shape[0]):
        temp_value = table_b_df.loc[i, attr]
        temp_num = max(temp_num, len(temp_value.split()))

    attr_maxnum_d[attr] = temp_num

print(attr_maxnum_d.items())

dict_items([('title', 51), ('authors', 47), ('venue', 17), ('year', 1)])


### 1.2 构建sentences embedding model

In [8]:
# import nltk
# nltk.download('punkt')

In [9]:
# 获取语料
attr_list = table_a_df.columns.values.tolist()[1:]
print(attr_list)

# 从dataFrame数据当中获取训练词向量模型的语料
all_sentences = []

# 获取table_a_df的语料
df_value = table_a_df[attr_list].values
for item in df_value:
    temp_list = list(item)
    temp_s = ', '.join(temp_list)
    all_sentences.append(temp_s)

# 获取table_b_df的语料
df_value = table_b_df[attr_list].values
for item in df_value:
    temp_list = list(item)
    temp_s = ', '.join(temp_list)
    all_sentences.append(temp_s)

print(len(all_sentences))
print(all_sentences[0])

['title', 'authors', 'venue', 'year']
66879
towards a cooperative transaction model   the cooperative activity model m rusinkiewicz   w klas   t tesch   j wфsch   p muth 1995, nan, vldb, nan


### 1.3 训练sentences model，并保存模型

In [10]:
from transformers import logging

logging.set_verbosity_warning()

In [11]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)
# polling 可以选择：cls, mean, max,三种，论文中说mean效果最好
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'mean')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Define a list with sentences (1k - 100k sentences)
train_sentences = all_sentences

# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

## Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=100,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 1e-5},
    show_progress_bar=True
)

# model.save('./tsdae-model50')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.
Some weights of the model checkpoint at bert-base-uncased were not used when initi

The following encoder weights were not tied to the decoder ['bert/pooler']


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/8360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/8360 [00:00<?, ?it/s]

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Iteration:   0%|          | 0/8360 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

Iteration:   0%|          | 0/8360 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
# 构造一个将所有句子，转换为tuple vector 的函数
def TupleEmbedding(all_sentences, sentence_embedding_model):
    '''
    all_sentences: 包含所有元组句子的列表
    sentence_embedding_model: 将句子转换为句子向量的模型
    founction：将所有的句子，转换为对应的sentence vector
    '''
    sentences_vector = []
    for sen in all_sentences:
        sentences_vector.append(sentence_embedding_model.encode(sen))
    return np.array(sentences_vector)

In [13]:
sentences_vector = TupleEmbedding(all_sentences, model)

# 先给train_df添加一个属性列similar:表示两个元组对应向量之间的余弦相似度
train_df['similar'] = 0.0

# 计算出train_df中所有元组对，对应的余弦相似度
for i in range(train_df.shape[0]):
    a_id = train_df.loc[i, 'ltable_id']
    b_id = train_df.loc[i, 'rtable_id']
    em_a = sentences_vector[a_id]
    em_b = sentences_vector[table_a_df.shape[0] + b_id]
    train_df.loc[i, 'similar'] = float(util.cos_sim(em_a, em_b))

train_df_0 = train_df[train_df['label'] == 0].copy(deep=True)
train_df_1 = train_df[train_df['label'] == 1].copy(deep=True)

In [14]:
# 保存X矩阵的内容
X = sentences_vector
with open(file_path + 'X_array0_100.txt', 'w') as fp:
  for item in X:
    temp = [str(i) for i in item]
    s = ' '.join(temp)
    fp.write(s + '\n')

In [15]:
train_df_0.describe()

,ltable_id,rtable_id,label,similar
count,14016.000000,14016.000000,14016.0,14016.000000
mean,1372.846104,31949.241296,0.0,0.317666
std,783.348707,18241.991871,0.0,0.141267
min,0.000000,25.000000,0.0,-0.064214
25%,661.000000,16558.000000,0.0,0.222211
50%,1407.000000,31948.000000,0.0,0.296421
75%,2005.000000,47486.500000,0.0,0.382581
max,2615.000000,64262.000000,0.0,0.936705


In [16]:
train_df_1.describe()

,ltable_id,rtable_id,label,similar
count,3207.000000,3207.000000,3207.0,3207.000000
mean,1313.052385,31933.857499,1.0,0.798131
std,711.339673,18667.178254,0.0,0.117142
min,0.000000,13.000000,1.0,0.295927
25%,746.000000,15594.000000,1.0,0.731348
50%,1331.000000,31582.000000,1.0,0.820134
75%,1857.000000,48139.500000,1.0,0.884171
max,2615.000000,64238.000000,1.0,1.000000
